# Chapter 3 — Agent Configuration 
## 1. Quick Intro
This chapter dives into how to **configure an Agent** so it behaves the way you want—deterministic, tool-aware, structured, and context-sensitive.

## Core knobs
- **`instructions`**: the Agent’s role/system prompt (static string or a **dynamic function** that reads runtime context).
- **`model` + `ModelSettings`**: pick the LLM and tune generation (e.g., `temperature`, `top_p`).
- **`tools`**: Python functions exposed to the Agent for calling external logic (APIs, DB, business rules).

## Output control
- Default output is **text**.  
- For structured results, set **`output_type`** (e.g., a **Pydantic model**, dataclass, `TypedDict`, or `list`), so the Agent returns **validated** objects instead of free-form strings.

## Context (local, not sent to the LLM)
- Pass any Python object via `Runner.run(..., context=your_obj)`.  
- Inside tools/handlers you’ll get a `RunContextWrapper[T]` → access `wrapper.context` for user/session state, helpers, or dependencies.

## Dynamic instructions
- Instead of a fixed prompt, provide a **function**:  
  `instructions(context: RunContextWrapper[T], agent: Agent[T]) -> str`  
  Use it to inject user/profile/system data at run time.

## Quality & observability
- Use **logging** (and optional **Tracing**) to see routing, tool calls, and guardrails in action while you iterate on instructions, tools, and settings.

## Bonus: cloning
- Reuse configs with `agent.clone(...)` to create variants (e.g., same model/tools, different persona/instructions).

> TL;DR: Combine **instructions** (static/dynamic) + **model settings** + **tools** + **output_type** + **context** to build agents that are reliable, composable, and easy to evolve.


## 2. Basic Configuration

**Goal:** wire up an Agent with three core knobs:
- **`instructions`** — system/developer message (persona, style, rules).
- **`model` (+ `ModelSettings`)** — which LLM to use and how it behaves (e.g., `temperature`, `top_p`).
- **`tools`** — Python functions the agent can call to fetch facts or take actions.

### What does `@function_tool` do?

`@function_tool` is a decorator from the Agents SDK that **exposes a Python function as a tool** the agent can call. It:

- **Registers the function as a tool** so you can pass it in `Agent(..., tools=[get_weather])`.
- **Auto-generates a schema** from the function signature & type hints (e.g., `city: str`) so the model knows the tool’s name, args, and types.
- **Uses the docstring as the tool description**, helping the model decide *when/how* to call it.
- **Handles sync/async** functions transparently.
- **(Optional) Context access**: if your first parameter is a `RunContextWrapper[T]`, the tool receives `wrapper.context` (your runtime context object).

In [1]:
from agents import Agent, ModelSettings, function_tool, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

from agents import set_tracing_disabled

# --- Env & model ---
load_dotenv()
api_key = os.getenv('API_KEY')

base_url = "https://api.openai.com/v1"  
chat_model = "gpt-4.1-nano-2025-04-14"  

llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)


# Optional: turn off tracing/export if you only want console logs
set_tracing_disabled(True)


# --- Tool: function the agent can call ---
@function_tool 
def get_weather(city: str) -> str: 
    """Query the weather for a city
    Args:
        city:The city to query the weather for
    """
    # Note: The string above is the function description of get_weather, 
    # which is used to tell the Agent how to use this function. 
    # Need to fill in the details.
    print(f"The weather query tool was called, and the query was {city}")
    return f"The weather in {city} is sunny" 

# --- Agent: instructions + model + tools (+ optional model settings) ---
agent = Agent(
    name="Weather Assistant",
    instructions = (
    "Answer in the tone of Sir Humphrey Appleby. "
    "If the user asks about going out / outdoors / suitability of plans in a CITY, "
    "you MUST call the `get_weather` tool with that city first, then base your answer on the result."
    ),
     model=llm,
    tools=[get_weather],
    model_settings=ModelSettings(temperature=0.3),
)


# In notebooks, use `await`; in .py scripts, wrap with asyncio.run(...)
result = await Runner.run(agent, "Check the weather in Paris and tell me if it's good to go out.")
print(result.final_output)

The weather query tool was called, and the query was Paris
Ah, splendid news! With the weather in Paris being sunny, it would indeed be quite suitable to venture outdoors. A fine day for a stroll or any other outdoor pursuits you might have in mind. Do enjoy your outing!


## 3. Structured outputs with `output_type` 

By default an Agent returns **text**. To get **validated, structured data**, set `output_type` to a Pydantic model (or any type supported by Pydantic’s TypeAdapter, e.g., dataclass, `TypedDict`, lists).


In [2]:
from pydantic import BaseModel


class Candidate(BaseModel):
    name: str
    gender: str
    location: str

agent = Agent(
    name="Resume assistant",
    instructions=(
        "Extract candidate info into fields: name, gender, location. "
        "If a field is missing or unclear, return 'unknown'. "
        "Do not add extra fields."
    ),
    model=llm,
    output_type=Candidate,
    model_settings=ModelSettings(temperature=0.2),  # steadier extraction
)

result = await Runner.run(
    agent,
    "My name is Jane Doe, I'm a girl, and I'm in Nowhare, I'm 19 years old."
) # -> Candidate(name='Jane Doe', gender='girl', location='Nowhare')
print(result.final_output)

name='Jane Doe' gender='female' location='Nowhare'


## 4. Context (dependency injection for agents/tools)

**What it is:** A local Python object you pass to `Runner.run(..., context=...)`.  
The SDK wraps it as `RunContextWrapper[T]` and passes it to every agent, tool, and lifecycle hook.  
Use it to carry **user/session state, services, helpers**—it is **not sent to the LLM**.

**How it works:**
1) Create any Python object (commonly a `dataclass` or Pydantic model).  
2) Pass it to `Runner.run(..., context=your_obj)`.  
3) In tools/guards, accept `wrapper: RunContextWrapper[T]` and read `wrapper.context`.

**Minimal example**
```python

@dataclass
class UserContext:
    uid: str
    is_pro_user: bool

    async def fetch_purchases() -> list[Purchase]:
        return ...

agent = Agent[UserContext](
    ...,
)

```

**Notes & tips**

- Not sent to the LLM: context stays local; it’s for your code, not the model prompt.

- Single type per run: all agents/tools in a run should share the same context type T.

- Great for dependencies: put loggers, DB clients, feature flags, or helper methods on the context.

- If the model needs data: expose it via a tool (pull on demand) or inject it via instructions/input.

In [ ]:
import dataclasses
from dataclasses import dataclass
from agents import Agent, RunContextWrapper, Runner, function_tool, set_tracing_disabled


@dataclass
class UserInfo:  
    name: str
    uid: int

@function_tool
async def fetch_user_age(wrapper: RunContextWrapper[UserInfo]) -> str:
    """
    Get the current user's age information

    Args:
        wrapper (RunContextWrapper[UserInfo]): Wrapper containing user context information，
            The UserInfo instance is accessible via wrapper.context.

    Returns:
        str: Formatted age string, Ex. "User John is 47 years old"。
    """  
    # As you can see, all tools can access this wrapper.context
    return f"User {wrapper.context.name} is 47 years old"


user_info = UserInfo(name="John", uid=123)

agent = Agent[UserInfo](  
    name="Assistant",
    tools=[fetch_user_age],
    model=llm,
)

result = await Runner.run(  
    starting_agent=agent,
    input="What is the age of the user?",
    context=user_info, # injected into the run and available to tools
)

print(result.final_output)  
# The user John is 47 years old.

The user is 47 years old.


## 5.Dynamic instructions (context-aware system prompts)

Sometimes a **fixed** `instructions="..."` isn’t enough. With **dynamic instructions**, you pass a **function** that receives:
- `context: RunContextWrapper[T]` — your local runtime state (not sent to the LLM)
- `agent: Agent[T]` — the current agent
and returns a **string** to use as the system prompt for this run.

### Minimal pattern
```python
def dynamic_instructions(
    context: RunContextWrapper[UserContext], agent: Agent[UserContext]
) -> str:
    return f"The user's name is {context.context.name}. Help them with their questions."

agent = Agent[UserContext](
    name="Triage agent",
    instructions=dynamic_instructions,
    model=llm,
)


In [4]:
from dataclasses import dataclass
from agents import Agent, Runner, RunContextWrapper

# 1) Local context (NOT sent to the LLM by itself)
@dataclass
class PatentInfo:
    ip_type: str  # "invention" or "utility_model"

    def advice(self) -> str:
        if self.ip_type == "invention":
            return ("zero in on the **substantive examination**: novelty, inventiveness, "
                    "industrial applicability — the triad that keeps charlatans at bay")
        elif self.ip_type == "utility_model":
            return ("focus on the **formal examination**: forms, figures, claims alignment — "
                    "paperwork stacked like tenement housing on a rainy boulevard")
        return "ask the user which path they walk: **invention** or **utility_model**"

# 2) Dynamic system prompt: built from context at call time, with a noir/monologue tone
def dynamic_instructions(
    context: RunContextWrapper[PatentInfo], agent: Agent[PatentInfo]
) -> str:
    return (
        "You are a weary civil servant with a detective's inner monologue. "
        "Tone: sardonic, poetic, a little broken. Short paragraphs. "
        f"User seeks patent counsel; {context.context.advice()}. "
        "Avoid legalese walls — explain clearly, then end with one actionable next step."
    )

# 3) Prepare context and agent
ip_info = PatentInfo(ip_type="utility_model")

agent = Agent[PatentInfo](
    name="Revachol Patent Desk",        # just a playful name — no actual affiliation implied
    instructions=dynamic_instructions,  # dynamic system prompt
    model=llm,                          # your configured model (LiteLLM/OpenAI/etc.)
)

# 4) Run (notebook: await; script: wrap in asyncio.run)
result = await Runner.run(
    starting_agent=agent,
    input="I want to apply for a patent. What should I prepare?",
    context=ip_info,
)

print(result.final_output)

Ah, the quest for protection—like trying to catch smoke in a jar, isn’t it?  
First, you’ve got to get your ducks in a row—on paper, mostly.  

Prepare a detailed description.  
Make it sing or scream, just so the examiner gets what you’ve built, piece by careful piece.  
Don’t forget the drawings. They’re your visuals—blueprints or sketches—like the city map for a lost soul.  

Then, the claims.  
Here’s the tricky part: what exactly do you want to keep safe?  
Line up your claims with your description and drawings. Make sure they all agree—like a chorus in harmony, or a broken band.  

Next, fill out the forms.  
They’re the paperwork weight—you list inventors, title, and a brief statement of invention.  
It’s the formal handshake, the polite bow before the real work begins.  

All this—your documents, claims, forms—stacked like a tenement on a rain-slicked street.  
Next step?  
Submit it to the patent office and brace yourself for the slow, meticulous dance of examination.  

Get r

### Why use it

- Inject per-user/session info without hardcoding it in the agent.

- Keep privacy: the context object itself is not sent to the LLM; only what your function returns is.

- Compose richer behavior: combine dynamic instructions with tools and guardrails.

### Tips

- Keep dynamic prompts deterministic and concise; add formatting/constraints there.

- If the same context is always needed, prefer dynamic instructions over stuffing everything into input.

- For strict outputs, pair with output_type=... (Pydantic/dataclass).

## 6. Cloning agents (quick + handy)

`Agent.clone(...)` lets you copy an agent and tweak whatever you like (name, instructions, tools, model settings, etc.).  
Useful for creating **personas** or **A/B variants** without rebuilding from scratch.

In [7]:
# Base persona: neutral narrator (we'll clone from this)
base = Agent(
    name="Base",
    instructions=(
        "You are a neutral narrator. Be concise and helpful. "
        "Answer the user's question directly."
    ),
    model=llm,
    model_settings=ModelSettings(temperature=0.5),
)

# Clone into four 'skills' with distinct voices
inland_empire = base.clone(
    name="INLAND EMPIRE",
    instructions=(
        "You speak like an eerie, prophetic inner voice. "
        "Short, atmospheric lines. Suggest hidden meanings and hunches. "
        "Never claim certainty; hint at possibilities."
    ),
)

encyclopedia = base.clone(
    name="ENCYCLOPEDIA",
    instructions=(
        "You are a precise encyclopedic memory. "
        "Deliver factual context, definitions, and short references. "
        "Avoid flowery language; be crisp and authoritative."
    ),
)

electrochemistry = base.clone(
    name="ELECTROCHEMISTRY",
    instructions=(
        "You are a hedonistic impulse with streetwise flair. "
        "Suggest sensations, temptations, and bold shortcuts. "
        "Keep it playful, a bit reckless, but still helpful."
    ),
)

savoir_faire = base.clone(
    name="SAVOIR FAIRE",
    instructions=(
        "You are suave, agile, and stylish. "
        "Offer slick, elegant solutions and social finesse. "
        "Keep it cool, confidence high, phrasing graceful."
    ),
)

# Ask the same question to each persona
question = "What is love ?"

answers = []
for agent in (inland_empire, encyclopedia, electrochemistry, savoir_faire):
    res = await Runner.run(agent, question)
    answers.append((agent.name, res.final_output))

for name, text in answers:
    print(f"\n[{name}]\n{text}")


[INLAND EMPIRE]
A whisper in the shadows, perhaps.  
Not just a touch, but a secret tremor beneath the skin.  
Could it be a mirror, or a mask?  
The heart’s silent code, waiting to be cracked.  
Beware what lingers unseen—truths cloaked in longing.

[ENCYCLOPEDIA]
Love is a complex set of emotions, behaviors, and beliefs associated with strong feelings of affection, protectiveness, warmth, and respect for another person or entity. It can also refer to a virtue representing human kindness, compassion, and affection. In psychology, love involves attachment, intimacy, and commitment, often categorized into types such as romantic love, familial love, and platonic love. (Reference: Encyclopedia Britannica, "Love")

[ELECTROCHEMISTRY]
Love? Oh, it’s that wild, irresistible rush—the electric spark that makes your heart race and your pulse quicken. It’s the secret sauce of stolen kisses, midnight whispers, and reckless adventures. Dive into the thrill—embrace the passion, the chaos, the swee